# Borrower reliability study

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a special system that evaluates the ability of a potential borrower to repay a loan to a bank.

## Exploring general information

In [1]:
# import required libraries
import pandas as pd
import seaborn
from IPython.display import display

In [2]:
# reading a data file and saving to data
data = pd.read_csv('data_banking.csv')

# getting information about a file
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Conclusion**

We get general data on the table, as well as look at its structure using the "head" method.

1. There are gaps in two columns
2. Data types correspond to the corresponding columns
3. Abnormal data is present (for example: 4 row column days_employed)

## Data preprocessing

### Pass processing

In [3]:
# Let's count the gaps
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# get median values for columns with gaps
days_median = data['days_employed'].median()
print('Median number of days worked', days_median)

income_median = data['total_income'].median()
print('Median number of income', income_median)

Median number of days worked -1203.369528770489
Median number of income 145017.93753253992


In [5]:
# count the ratio of gaps to the number of elements in the column
miss_ratio_days = (data['days_employed'].isna().mean())
print(miss_ratio_days)
miss_ratio_income = (data['total_income'].isna().mean())
print(miss_ratio_income)

0.10099883855981417
0.10099883855981417


In [6]:
# populate income data with the median value
data['total_income'] = data['total_income'].fillna(income_median)

# check the changes made
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

**Conclusion**

Based on the data obtained, we see that there are gaps in two columns:

    1. `days_employed`
    2. `total_income`
The gaps in these columns can be for the following reasons:

    1. Lack of work / freelance / passive income / errors when filling out or uploading data
    2. Unwillingness to indicate your income / errors when filling in or uploading data
It is worth noting that the number of gaps in these columns is the same, and is 10.1% of the total number of values.

Since income is a quantitative variable, the gaps should be filled with the median value, since, unlike the mean, it divides the sample strictly in half, taking into account outliers in the form of anomalously small or large values.

### Replacing the data type

In [7]:
# replace float64 type in total_income and days_employed with int64
data['total_income'] = data['total_income'].astype('int')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### Duplicate Handling

Let's review the data in columns to identify explicit and implicit duplicates

In [8]:
# unique values of the 'children' column
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

Seems to be:
    
- data -1 is a misspelled 1
- 20 are misspelled 2

We will make the appropriate corrections

In [9]:
# making corrections and checking results
data.loc[(data['children'] == 20), 'children'] = 2
data.loc[(data['children'] == -1), 'children'] = 1
data['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [10]:
# unique values of the 'days_employed' column
data['days_employed'].sort_values().unique()

array([-18388.94990057, -17615.56326563, -16593.47281726, ...,
       401715.81174889, 401755.40047533,             nan])

When checking the values, 2 problems were found:
1. Negative values of seniority. Most likely caused by a calculation error, when the current date was subtracted from the start date, which led to negative values
2. Abnormally high values of seniority

In [11]:
# Correction of negative seniority values
rows = (data['days_employed'] < 0)
data.loc[rows, 'days_employed'] = abs(data['days_employed'])
data['days_employed'].sort_values()

17437    24.141633
8336     24.240695
6157     30.195337
9683     33.520665
2127     34.701045
           ...    
21489          NaN
21495          NaN
21497          NaN
21502          NaN
21510          NaN
Name: days_employed, Length: 21525, dtype: float64

In [12]:
# look at abnormal values based on the assumption that the work experience does not exceed (65 - 18 years * 365 days)
data[data['days_employed'] > 17500]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем


In [13]:
# Presumably all clients with incorrect seniority value are pensioners
# Check it out
data[(data['days_employed'] > 17500) & (data['income_type'] == 'пенсионер')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем


The assumption was justified, so you can replace these data with the maximum value for seniority.
Or assume that this is the result of an error in the calculation of days, and the resulting values should be divided by 24 hours

In [14]:
# Assume the second option and make changes
rows = ((data['days_employed'] > 17500))
data.loc[rows, 'days_employed'] = data['days_employed'] / (24)
data['days_employed'].sort_values().unique()

array([   24.14163324,    24.24069479,    30.19533716, ...,
       16738.15882287, 16739.80835314,            nan])

In [15]:
# now find the median value for the days of work experience and fill in the gaps with it
days_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_median)
data['days_employed'].sort_values()

17437       24.141633
8336        24.240695
6157        30.195337
9683        33.520665
2127        34.701045
             ...     
7794     16735.993752
2156     16736.436110
7664     16736.462226
10006    16738.158823
6954     16739.808353
Name: days_employed, Length: 21525, dtype: float64

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


Thus, the anomalous values in the seniority column are accounted for.

In [17]:
# Check the values of the 'dob_years' column
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

In [18]:
# Specify the number of 0 values
data[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,2194.207271,0,среднее,1,женат / замужем,0,F,сотрудник,0,145017,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


In [19]:
# Fill 0 values with the median
days_median = data['dob_years'].median()
rows = ((data['dob_years'] == 0))
data.loc[rows, 'dob_years'] = days_median
data['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75], dtype=int64)

In [20]:
# Check the values of the 'dob_years' column and get rid of duplicates
data['education'] = data['education'].str.lower()
data = data.drop_duplicates()
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [21]:
# Checking the learning column
data['education_id'].sort_values().unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [22]:
# Checking the 'family_status' column
data['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [23]:
# Check column 'family_status_id'
data['family_status_id'].sort_values().unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [24]:
# Checking the 'gender' column
data['gender'].sort_values()

0          F
12840      F
12841      F
12844      F
12845      F
        ... 
11053      M
11054      M
3841       M
10743      M
10701    XNA
Name: gender, Length: 21453, dtype: object

In [25]:
data[data['gender'] == 'XNA']
data = data.drop(labels = [10701],axis = 0)
data['gender'].sort_values()

0        F
12840    F
12841    F
12844    F
12845    F
        ..
11054    M
11055    M
11056    M
3831     M
10743    M
Name: gender, Length: 21452, dtype: object

In [26]:
# Check column 'income_type'
data['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

**Conclusion**

Outliers and duplicates in various columns were processed.

To handle implicit duplicates, a string reduction method was used.
The median value was used to fill in zero age values.
The value of XNA in gender remained unchanged, since it does not contribute much to the sample. Removing this line is irrational.

The largest number of questions arose with the `days_employed` column. Most likely, there is an error in the algorithm for filling this variable

### Lemmatization

In [27]:
# Create a function to handle the 'purpose' column
def rename_purpose(row):
    purpose = row['purpose']
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'ремонт' in purpose:
        return 'операции с недвижимостью'
data['purpose_category'] = data.apply(rename_purpose, axis=1)
display(data.head(50))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью


In [28]:
# Create a dataframe for unique learning IDs
uniq_education_id = pd.DataFrame({'education' : ['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], 'education_ID' : [0, 1, 2, 3, 4]})
uniq_education_id.head()

,education,education_ID
0,высшее,0
1,начальное,1
2,неоконченное высшее,2
3,среднее,3
4,ученая степень,4


In [29]:
# Create a dataframe for unique marital status identifiers
uniq_family_id = pd.DataFrame({'family_status' : ['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], 'family_ID' : [0, 1, 2, 3, 4]})
uniq_family_id.head()

,family_status,family_ID
0,Не женат / не замужем,0
1,в разводе,1
2,вдовец / вдова,2
3,гражданский брак,3
4,женат / замужем,4


In [30]:
# remove the row data from the original dataframe
del data['education']
del data['family_status']
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21452 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     21452 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education_id      21452 non-null  int64  
 4   family_status_id  21452 non-null  int64  
 5   gender            21452 non-null  object 
 6   income_type       21452 non-null  object 
 7   debt              21452 non-null  int64  
 8   total_income      21452 non-null  int32  
 9   purpose           21452 non-null  object 
 10  purpose_category  21452 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(4)
memory usage: 1.9+ MB


**Conclusion**

The 'purpose' column has been lemmatized.

Created for marital status and education dataframe containing unique identifiers.

### Data categorization

In [31]:
# Create a function to categorize income
def rename_purpose(row):
    purpose = row['total_income']
    if purpose > 1000001:
        return 'A'
    if 200001 <= purpose <= 1000000:
        return 'B'
    if 50001 <= purpose <= 200000:
        return 'C'
    if 30001 <= purpose <= 50000:
        return 'D'
    if purpose <= 30000:
        return 'E'
data['total_income_category'] = data.apply(rename_purpose, axis=1)
display(data.head(50))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,получение образования,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью,C


Income data is categorized 

## Answers on questions

- Is there a relationship between having children and repaying the loan on time?

In [32]:
# use a pivot table to answer the question
data_pivot_child = data.pivot_table(index='purpose_category', columns='children', values='debt', aggfunc='sum')
data_pivot_child = data_pivot_child.fillna(value=0)
data_pivot_child

children,0,1,2,3,4,5
purpose_category,,,,,,
операции с автомобилем,243,104,50,5,1,0
операции с недвижимостью,476,200,90,13,3,0
получение образования,229,90,47,4,0,0
проведение свадьбы,115,51,15,5,0,0


In [33]:
# Find the conversion
age = (data['children'].sort_values().unique())
debt = [0, 1]
ratio = []

# Calculation of the ratio of successful loan repayments to the total number of debtors by the number of children
for age_value in age:
        norm = (data.loc[(data['children'] == age_value) & (data['debt'] == 0)])
        all = data.loc[(data['children'] == age_value)]
        try:
            ratio.append(int(norm['debt'].count() / all['debt'].count()*100))
        except:
            print("Division by zero! Check data!")

ratio_purpose = pd.DataFrame({'ratio_success %': ratio, 'children': age, 'all_debtors' : data['children'].value_counts()})
ratio_purpose

,ratio_success %,children,all_debtors
0,92,0,14089
1,90,1,4855
2,90,2,2128
3,91,3,330
4,90,4,41
5,100,5,9


Based on the data obtained, the following conclusion can be drawn:

People with fewer children are more likely to be in arrears. The largest number of delinquencies relates to real estate transactions, followed by car transactions.

- Is there a relationship between marital status and loan repayment on time?

In [34]:
data_pivot_family = data.pivot_table(index='purpose_category', columns='family_status_id', values='debt', aggfunc='sum')
data_pivot_family = data_pivot_family.fillna(value=0)
data_pivot_family

family_status_id,0,1,2,3,4
purpose_category,,,,,
операции с автомобилем,229.0,51.0,20.0,21.0,82.0
операции с недвижимостью,486.0,91.0,28.0,47.0,130.0
получение образования,216.0,60.0,15.0,17.0,62.0
проведение свадьбы,0.0,186.0,0.0,0.0,0.0


In [35]:
# Find the conversion
family = (data['family_status_id'].sort_values().unique())
debt = [0, 1]
ratio = []

# Calculation of the ratio of successful loan repayments to the total number of debtors by marital status
for family_value in family:
        norm = (data.loc[(data['family_status_id'] == family_value) & (data['debt'] == 0)])
        all = data.loc[(data['family_status_id'] == family_value)]
        try:
            ratio.append(int(norm['debt'].count() / all['debt'].count()*100))
        except:
            print("Division by zero! Check data!")

ratio_purpose = pd.DataFrame({'ratio_success %': ratio, 'family_status_id': family, 'all_debtors' : data['family_status_id'].value_counts()})
ratio_purpose

,ratio_success %,family_status_id,all_debtors
0,92,0,12339
1,90,1,4149
4,93,2,2810
3,92,3,1195
2,90,4,959


Based on the data obtained, the following conclusion can be drawn:

The most prone to delay are people who are in registered or civil marriages. The least prone to non-payment are people who are divorced and have lost their loved ones. Again, housing debt gets the most delinquency.

For obvious reasons, only those who are in common-law unions can be behind on "wedding arrangements" payments.

- Is there a relationship between the level of income and repayment of the loan on time?

In [36]:
data_pivot_incom = data.pivot_table(index='purpose_category', columns='total_income_category', values='debt', aggfunc='sum')
data_pivot_incom = data_pivot_incom.fillna(value=0)
data_pivot_incom

total_income_category,A,B,C,D,E
purpose_category,,,,,
операции с автомобилем,0,86,307,10,0
операции с недвижимостью,1,172,601,6,2
получение образования,1,70,297,2,0
проведение свадьбы,0,28,155,3,0


In [37]:
# Find the conversion
incom = (data['total_income_category'].sort_values().unique())
debt = [0, 1]
ratio = []

# Calculation of the ratio of successful loan repayments to the total number of debtors by goals for the loan
for incom_value in incom:
        norm = (data.loc[(data['total_income_category'] == incom_value) & (data['debt'] == 0)])
        all = data.loc[(data['total_income_category'] == incom_value)]
        try:
            ratio.append(int(norm['debt'].count() / all['debt'].count()*100))
        except:
            print("Division by zero! Check data!")

ratio_purpose = pd.DataFrame({'ratio_success %': ratio, 'total_income_category': incom, 'all_debtors' : data['total_income_category'].value_counts()})
ratio_purpose

,ratio_success %,total_income_category,all_debtors
C,92,A,16015
B,92,B,5040
D,91,C,350
A,94,D,25
E,90,E,22


Based on the data obtained, the following conclusion can be drawn:

People with income category A have single delays, as do people with income category E. The largest number of delays relate to income categories B and C.

- How do different purposes of the loan affect its repayment on time?

In [38]:
data_pivot_purpose = data.pivot_table(columns='purpose_category', values='debt', aggfunc='sum')
data_pivot_purpose = data_pivot_purpose.fillna(value=0)
data_pivot_purpose

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,403,782,370,186


In [39]:
# Find the conversion
credit_purpose = (data['purpose_category'].sort_values().unique())
debt = [0, 1]
ratio = []

# Calculation of the ratio of successful loan repayments to the total number of debtors by goals for the loan
for purpose_value in credit_purpose:
        norm = (data.loc[(data['purpose_category'] == purpose_value) & (data['debt'] == 0)])
        all = data.loc[(data['purpose_category'] == purpose_value)]
        try:
            ratio.append(int(norm['debt'].count() / all['debt'].count()*100))
        except:
            print("Division by zero! Check data!")

ratio_purpose = pd.DataFrame({'ratio_success %': ratio, 'purpose_category': credit_purpose, 'all_debtors' : data['purpose_category'].value_counts()})
ratio_purpose

,ratio_success %,purpose_category,all_debtors
операции с недвижимостью,90,операции с автомобилем,10810
операции с автомобилем,92,операции с недвижимостью,4306
получение образования,90,получение образования,4013
проведение свадьбы,91,проведение свадьбы,2323


Based on the data obtained, the following conclusion can be drawn:

Real estate transactions contain the largest number of delinquencies. Transactions with a car and education have an average value for delinquencies

## General conclusion

In this work, 4 hypotheses were tested on the repayment of the debt on time, depending on some data of the borrowers.

Data preprocessing, lemmatization and categorization were performed.

Relative and absolute indicators are determined, goals with the largest number of delays are determined.

The results for some values have little statistics - as a result, the result is less reliable.